In [3]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from glue import BlindNavigator
from utility import read_video
from PIL import Image
import numpy as np
import cv2
import time

class Visor(object):
    def __init__(self, form = 'OFFLINE'):
        self.form = form
        self.handler = None
        self.handler2 = None
        self.rescale = True
        self.output_size = (600, 400)
        self.im_size = self.output_size


        
    def initializer(self):
        if self.form == 'OFFLINE':
            self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
            writer = None
            (h, w) = (None, None)
            self.handler = writer 
            
        else:
            fig = plt.figure(figsize=(16, 10))
            ax = fig.add_axes([0,0,1,1])

            plt.ion()
            fig.show()
            fig.canvas.draw()
            self.handler = fig
            self.handler2 = ax

            
    def inliner(self, im, save_path='./test.avi'):
        self.im_size = (im.shape[1], im.shape[0])
        (w, h) =  self.output_size

        if self.rescale:
            im = cv2.resize(im, (w, h),interpolation=cv2.INTER_CUBIC)
        else:
            (h, w) = im.shape[:2]

        if self.form == 'OFFLINE' and self.handler is None:
            self.handler = cv2.VideoWriter(save_path, self.fourcc, 30, (w, h), True)
        elif self.form != 'OFFLINE':
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        return im

    def drawer(self, im):
        if self.form == 'OFFLINE':
            self.handler.write(im)
        else:
            self.handler2.imshow(im)
            self.handler.canvas.draw()
            self.handler2.clear()
            
    def rescale_box(self, bbox):
        if self.rescale:
            bbox = np.array([bbox[0] / self.im_size[0] * self.output_size[0], \
                             bbox[1] / self.im_size[1] * self.output_size[1], \
                             bbox[2] / self.im_size[0] * self.output_size[0], \
                             bbox[3] / self.im_size[1] * self.output_size[1]])
        bbox = bbox.astype(int)
        return bbox
    
debuger = 'OFFLINE'
def test_det_track():
    video_path = 'data/IMG_1051.MOV'
    save_path = 'IMG_1051_det_track.avi'

    visor = Visor(debuger)
    visor.initializer()

    navigator = BlindNavigator()
    timestamps = [time.time()]
    for data in read_video(video_path):
        timestamps.append(time.time())

        traffic_lights, detected_obstacles = navigator.detect_traffic_light(data)
        timestamps.append(time.time())
        print('Processing time: %.3f'%(timestamps[-1] - timestamps[-2]))
#         data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        data = visor.inliner(data, save_path)
        for (id_, bbox) in traffic_lights:
            bbox = visor.rescale_box(bbox)
            assert  id_ < len(navigator.traffic_light_pool.trackers), 'id_ not valid'
#             bbox = [bbox[0] / data.shape[1], bbox[1] / data.shape[0], bbox[2] / data.shape[1], bbox[3] / data.shape[0]]
#             print(bbox)
#             p = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], fill=False, alpha=1)
#             ax.add_patch(p)
            cv2.rectangle(data, tuple(bbox[:2]),tuple(bbox[2:4]), [255, 0, 0], 3)
        visor.drawer(data)

def test_light_classifier(video_path, save_path):


    visor = Visor(debuger)
    visor.initializer()
    navigator = BlindNavigator()
    for data in read_video(video_path):
        traffic_lights, detected_obstacles = navigator.detect_traffic_light(data)
        light_states = navigator.color_classify_by_boxes(data, [light[1] for light in traffic_lights])
        
        data = visor.inliner(data, save_path)
        for (id_, bbox), state in zip(traffic_lights, light_states):
            
            bbox = visor.rescale_box(bbox)

            assert  id_ < len(navigator.traffic_light_pool.trackers), 'id_ not valid'
#             bbox = [bbox[0] / data.shape[1], bbox[1] / data.shape[0], bbox[2] / data.shape[1], bbox[3] / data.shape[0]]
#             print(bbox)
#             p = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], fill=False, alpha=1)
#             ax.add_patch(p)
            cv2.rectangle(data, tuple(bbox[:2]),tuple(bbox[2:4]), [255, 0, 0], 3)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(data, '%s'%(state), (bbox[0]-1,bbox[1]-1), font, 1, color=(255,255,0), thickness=2)
        visor.drawer(data)

def test_zebra_contours(video_path, save_path):

    visor = Visor(debuger)
    visor.initializer()
    navigator = BlindNavigator()
    for data in read_video(video_path):
        p_lights, detected_obstacles = navigator.executor(data)
        data = visor.inliner(data, save_path)
        if p_lights is not None:
            bbox, state  = p_lights.get_bbox(), p_lights.get_state()
            print(bbox)
            bbox = visor.rescale_box(bbox)
            print(bbox)
#             assert  id_ < len(navigator.traffic_light_pool.trackers), 'id_ not valid'
            cv2.rectangle(data, tuple(bbox[:2]),tuple(bbox[2:4]), [255, 0, 0], 3)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(data, '%s'%(state), (bbox[0]-1,bbox[1]-1), font, 1, color=(255,255,0), thickness=2)

        for j in navigator.zebra_contours:
            bx, by, bw, bh = cv2.boundingRect(j)
            [bx, by, bw, bh] = visor.rescale_box([bx, by, bw, bh])
            cv2.rectangle(data, (bx, by), (bx + bw, by + bh), (180, 237, 167), -1)
        visor.drawer(data)


if __name__ == '__main__':
    video_path = 'data/IMG_9033.m4v'
    save_path = 'IMG_9033_zebra_track.avi'
    test_zebra_contours(video_path, save_path)

keras_yolo3/model_data/yolo.h5 model, anchors, and classes loaded.
get frame 0
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 1
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 2
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 3
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 4
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 5
Detected 0 traffic lights, in total 0 traffic lights, 1 obstacles
[]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 0
get frame 6
Detected 0 traffic li

Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 20, Forward Max Pedestrain Light vote 21
Tracker 0 has been determined as pedestrain light 21 times 
[661, 261, 679, 297]
[309 145 318 165]
get frame 43
Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[2]
Cur Max Pedestrain Light vote 21, Forward Max Pedestrain Light vote 21
[660, 262, 679, 299]
[309 145 318 166]
get frame 44
Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[2]
Cur Max Pedestrain Light vote 21, Forward Max Pedestrain Light vote 21
[660, 262, 680, 300]
[309 145 318 166]
get frame 45
Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[2]
Cur Max Pedestrain Light vote 21, Forward Max Pedestrain Light vote 21
[659, 262, 679, 300]
[308 145 318 166]
get frame 46
Detected 1 traffic lights, in total 1 traffic lights, 3 obstacles
[2]
Cur Max Pedestrain Light vote 21, Forward Max Pedestrain Light vote 21
[659, 262, 678, 300]
[308 1

Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 43, Forward Max Pedestrain Light vote 44
Tracker 0 has been determined as pedestrain light 44 times 
[659, 261, 678, 301]
[308 145 317 167]
get frame 73
Detected 0 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 44, Forward Max Pedestrain Light vote 45
Tracker 0 has been determined as pedestrain light 45 times 
[660.4344994079441, 263.88305018523107, 677.4344994079441, 296.88305018523107]
[309 146 317 164]
get frame 74
Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 45, Forward Max Pedestrain Light vote 46
Tracker 0 has been determined as pedestrain light 46 times 
[660, 261, 680, 302]
[309 145 318 167]
get frame 75
Detected 1 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 46, Forward Max Pedestrain Light vote 47
Tracker 0 has been determined as pedestrai

[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 68
Tracker 0 has been determined as pedestrain light 68 times 
[675.4228486259667, 273.83355145609835, 692.4228486259667, 306.83355145609835]
[316 152 324 170]
get frame 103
Detected 0 traffic lights, in total 1 traffic lights, 1 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 69
Tracker 0 has been determined as pedestrain light 69 times 
[675.9058204017992, 274.615203340867, 692.9058204017992, 307.615203340867]
[316 152 324 170]
get frame 104
Detected 0 traffic lights, in total 1 traffic lights, 1 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 70
Tracker 0 has been determined as pedestrain light 70 times 
[676.2280718968785, 275.02028978587373, 693.2280718968785, 308.02028978587373]
[316 152 324 171]
get frame 105
Detected 1 traffic lights, in total 1 traffic lights, 1 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 

Detected 0 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 92
Tracker 0 has been determined as pedestrain light 92 times 
[681.1269583581708, 292.30421621443634, 698.1269583581708, 325.30421621443634]
[319 162 327 180]
get frame 131
Detected 0 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 93
Tracker 0 has been determined as pedestrain light 93 times 
[680.8384655697314, 291.4481312501886, 697.8384655697314, 324.4481312501886]
[319 161 327 180]
get frame 132
Detected 0 traffic lights, in total 1 traffic lights, 2 obstacles
[1]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 94
Tracker 0 has been determined as pedestrain light 94 times 
[680.8703758915537, 291.63905490334605, 697.8703758915537, 324.63905490334605]
[319 162 327 180]
get frame 133
Detected 0 traffic lights, in total 1 traffic lights, 2 obstacles
[1]

...

Detected 1 traffic lights, in total 2 traffic lights, 6 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[606.6387710817435, 247.62688727457828, 624.4887710817435, 282.27688727457826]
[284 137 292 156]
get frame 1281
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[605.955589364791, 247.90662313069944, 623.805589364791, 282.55662313069945]
[284 137 292 156]
get frame 1282
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[605.1016438954082, 248.93253852953734, 622.9516438954082, 283.5825385295373]
[283 138 292 157]
get frame 1283
Detected 1 traffic lights, in total 2 traffic lights

Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[578.3258982948329, 253.4361006148914, 597.0683982948328, 289.8186006148914]
[271 140 279 161]
get frame 1307
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[577.9032217497357, 252.96914138180063, 596.6457217497357, 289.3516413818006]
[270 140 279 160]
get frame 1308
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[577.5591937929772, 253.82734826513743, 596.3016937929772, 290.20984826513745]
[270 141 279 161]
get frame 1309
Detected 1 traffic lights, in total 2 traffic lights

Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[545.066861292772, 299.4026528894343, 564.746486292772, 337.6042778894343]
[255 166 264 187]
get frame 1333
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[544.4174841218436, 301.28010890689006, 564.0971091218436, 339.48173390689004]
[255 167 264 188]
get frame 1334
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[543.4816397613786, 304.1491483251756, 563.1612647613786, 342.35077332517557]
[254 168 263 190]
get frame 1335
Detected 1 traffic lights, in total 2 traffic lights, 

Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[503.5659228986676, 335.6298718070626, 523.2455478986676, 373.8314968070626]
[236 186 245 207]
get frame 1360
Detected 1 traffic lights, in total 2 traffic lights, 3 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[503.4470553197295, 334.2016679041887, 523.1266803197295, 372.4032929041887]
[235 185 245 206]
get frame 1361
Detected 1 traffic lights, in total 2 traffic lights, 4 obstacles
[2, 2]
Cur Max Pedestrain Light vote 0, Forward Max Pedestrain Light vote 410
Tracker 0 has been determined as pedestrain light 410 times 
[503.8899810265163, 334.92888336034054, 523.5696060265163, 373.1305083603405]
[236 186 245 207]
get frame 1362
Detected 1 traffic lights, in total 2 traffic lights, 

TypeError: 'NoneType' object is not subscriptable